In [9]:
import numpy as np
import cv2
import imutils
import matplotlib.pyplot as plt
import pandas as pd
import json

In [10]:
df = pd.read_csv("HandInfo.csv", index_col="imageName")
df

,id,age,gender,skinColor,accessories,nailPolish,aspectOfHand,irregularities
imageName,,,,,,,,
Hand_0000002.jpg,0,27,male,fair,0,0,dorsal right,0
Hand_0000003.jpg,0,27,male,fair,0,0,dorsal right,0
Hand_0000004.jpg,0,27,male,fair,0,0,dorsal right,0
Hand_0000005.jpg,0,27,male,fair,0,0,dorsal right,0
Hand_0000006.jpg,0,27,male,fair,0,0,dorsal right,0
...,...,...,...,...,...,...,...,...
Hand_0011740.jpg,1589,22,female,fair,0,0,palmar left,0
Hand_0011741.jpg,1589,22,female,fair,0,0,palmar left,0
Hand_0011742.jpg,1589,22,female,fair,0,0,palmar left,0


In [11]:
# Drop wrongly labelled data
df.drop(df.loc["Hand_0006393.jpg":"Hand_0006408.jpg"].index, inplace=True)
df = df[(df.irregularities == 0) & ((df.aspectOfHand == "palmar right") | (df.aspectOfHand == "palmar left"))]
df

,id,age,gender,skinColor,accessories,nailPolish,aspectOfHand,irregularities
imageName,,,,,,,,
Hand_0000038.jpg,0,27,male,fair,1,0,palmar left,0
Hand_0000039.jpg,0,27,male,fair,1,0,palmar left,0
Hand_0000040.jpg,0,27,male,fair,1,0,palmar left,0
Hand_0000041.jpg,0,27,male,fair,1,0,palmar left,0
Hand_0000042.jpg,0,27,male,fair,1,0,palmar left,0
...,...,...,...,...,...,...,...,...
Hand_0011740.jpg,1589,22,female,fair,0,0,palmar left,0
Hand_0011741.jpg,1589,22,female,fair,0,0,palmar left,0
Hand_0011742.jpg,1589,22,female,fair,0,0,palmar left,0


In [38]:
%matplotlib inline

def getBbox(image, imageName):
        bin = cv2.inRange(image, (190, 190, 190), (255, 255,255))
        cv2.bitwise_not(bin, bin)
        cnts = cv2.findContours(bin.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        cnts = sorted(cnts, key = cv2.contourArea, reverse = True)

        rect = cv2.boundingRect(cnts[0])
        image = cv2.rectangle(image, rect, (0,0,0), 2)
        
        cv2.circle(image,(int(rect[0]),int(rect[1])), 8, (0,255,0), -1)
        cv2.imwrite(f"preprocessed-hands/{imageName}", cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
        # return a tuple of (x, y, width, height)
        return rect

images = df.index.to_numpy()
# images = ["Hand_0000038.jpg", "Hand_0000039.jpg", "Hand_0000040.jpg"] # ONLY FOR TESTING

bboxes = {
     "palmar right": {},
     "palmar left": {}
}

for imageName in images:
    image = cv2.imread(f"data/{imageName}") 
    image = cv2.rotate(image, cv2.ROTATE_180)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    bboxes[df.loc[imageName].aspectOfHand][imageName] = getBbox(image, imageName)

print(bboxes)
with open('bboxes.json', 'w', encoding='utf-8') as f:
    json.dump(bboxes, f, ensure_ascii=False, indent=4)


{'palmar right': {}, 'palmar left': {'Hand_0000038.jpg': (176, 254, 680, 946), 'Hand_0000039.jpg': (75, 250, 990, 950), 'Hand_0000040.jpg': (153, 259, 974, 941)}}
